<a href="https://colab.research.google.com/github/dlsun/pods/blob/master/10-Textual-Data/10.2%20The%20Vector%20Space%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10.2 The Vector Space Model

In the previous section, we saw how a single document could be converted into a _bag of words_ (or, more precisely, a bag of $n$-grams) representation. In this section, we go one step further, converting an entire corpus of documents into tabular data.

## Term Frequencies

The bag of words representation gives us a mapping between words and their counts, such as `{..., "am": 3, "i": 71, "sam": 6, ...}`. To turn the bag of words into a vector of numbers, we can simply take the word counts, as follows:

| ... | i  | am | sam | ... |
|-----|----|----|-----|-----|
| ... | 71 |  3 |  6  | ... |

If we do this for each document in the corpus, and stack the rows, we obtain a table of numbers called the _term-frequency matrix_. 

|        | ... | i  | am | sam | ... |
|--------|-----|----|----|-----|-----|
|**green_eggs_and_ham**| ... | 71 |  3 |  6  | ... |
|**cat_in_the_hat**| ... | 59 | 0 | 0 | ... |
|**fox_in_socks**| ... | 13 | 0 | 0 | ... |
|...|...|...|...|...|...|
|**one_fish_two_fish**| ... | 51 | 3 | 0 | ... |

The columns are all words (or _terms_) that appear in the corpus, which collectively make up the _vocabulary_. The idea of representing documents by a vector of numbers is called the _vector space model_.

### Implementation from Scratch

Let's obtain the term-frequency matrix for the Dr. Seuss books. First, we read in the data.

In [1]:
import pandas as pd
import requests

seuss_dir = "http://dlsun.github.io/pods/data/drseuss/"
seuss_files = [
    "green_eggs_and_ham.txt", "cat_in_the_hat.txt", "fox_in_socks.txt",
    "hop_on_pop.txt", "horton_hears_a_who.txt", "how_the_grinch_stole_christmas.txt",
    "oh_the_places_youll_go.txt", "one_fish_two_fish.txt"
]

docs_seuss = pd.Series()
for file in seuss_files:
    response = requests.get(seuss_dir + file, "r")
    docs_seuss[file[:-4]] = response.text

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


Now we apply the bag of words representation to the normalized text.

In [2]:
from collections import Counter

bag_of_words = (
    docs_seuss.
    str.lower().                  # convert all letters to lowercase
    str.replace("[^\w\s]", " ").  # replace non-alphanumeric characters by whitespace
    str.split()                   # split on whitespace
).apply(Counter)

bag_of_words

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


green_eggs_and_ham                {'i': 84, 'am': 16, 'sam': 19, 'that': 3, 'do'...
cat_in_the_hat                    {'the': 97, 'sun': 2, 'did': 10, 'not': 41, 's...
fox_in_socks                      {'fox': 17, 'socks': 19, 'box': 7, 'knox': 17,...
hop_on_pop                        {'up': 6, 'pup': 8, 'is': 12, 'cup': 4, 'in': ...
horton_hears_a_who                {'on': 21, 'the': 97, 'fifteenth': 1, 'of': 39...
how_the_grinch_stole_christmas    {'every': 5, 'who': 18, 'down': 10, 'in': 17, ...
oh_the_places_youll_go            {'congratulations': 1, 'today': 2, 'is': 7, 'y...
one_fish_two_fish                 {'one': 14, 'fish': 12, 'two': 4, 'red': 2, 'b...
dtype: object

To turn this into a term-frequency matrix, we need to make a `DataFrame` out of it, where each column represents a word and each row a document---and each entry is the count of that word in the document.

In [4]:
tf = pd.DataFrame(list(bag_of_words),index=bag_of_words.index)
tf

,i,am,sam,that,do,not,like,you,green,eggs,...,zeds,upon,heads,haircut,wave,swish,gack,park,clark,zeep
green_eggs_and_ham,84,16.0,19.0,3,36.0,82,44.0,34,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cat_in_the_hat,59,NaN,NaN,25,25.0,41,14.0,34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fox_in_socks,13,NaN,NaN,6,8.0,1,1.0,8,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hop_on_pop,2,1.0,NaN,5,NaN,2,6.0,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
horton_hears_a_who,43,1.0,NaN,36,7.0,7,NaN,47,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
how_the_grinch_stole_christmas,16,NaN,NaN,16,4.0,2,2.0,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oh_the_places_youll_go,6,NaN,NaN,12,4.0,9,1.0,85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
one_fish_two_fish,51,3.0,NaN,1,12.0,10,21.0,24,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0


In [7]:
import numpy as np
np.sum(np.sum(tf.isnull()))/tf.size

0.7836715867158671

This matrix is full of missing numbers. A missing number means that the word did not appear in that document. In other words, a count of `NaN` really means a count of 0. So it makes sense in this situation to replace the `NaN`s by 0s.

In [8]:
tf = tf.fillna(0)
tf

,i,am,sam,that,do,not,like,you,green,eggs,...,zeds,upon,heads,haircut,wave,swish,gack,park,clark,zeep
green_eggs_and_ham,84,16.0,19.0,3,36.0,82,44.0,34,10.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cat_in_the_hat,59,0.0,0.0,25,25.0,41,14.0,34,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fox_in_socks,13,0.0,0.0,6,8.0,1,1.0,8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hop_on_pop,2,1.0,0.0,5,0.0,2,6.0,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
horton_hears_a_who,43,1.0,0.0,36,7.0,7,0.0,47,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
how_the_grinch_stole_christmas,16,0.0,0.0,16,4.0,2,2.0,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oh_the_places_youll_go,6,0.0,0.0,12,4.0,9,1.0,85,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
one_fish_two_fish,51,3.0,0.0,1,12.0,10,21.0,24,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0


In [9]:
import numpy as np
np.sum(np.sum(tf.isnull()))/tf.size

0.0

### Implementation using `scikit-learn`

We could have also used the `CountVectorizer` in `scikit-learn` to obtain the term-frequency matrix. This vectorizer is fit to a list of the documents in the corpus. By default, it converts all letters to lowercase and strips punctuation, although this behavior can be customized using the `strip_accents=` and `lowercase=` parameters. 

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit(docs_seuss) # This determines the vocabulary.
tf_sparse = vec.transform(docs_seuss)

tf_sparse

<8x1344 sparse matrix of type '<class 'numpy.int64'>'
	with 2308 stored elements in Compressed Sparse Row format>

Notice that `CountVectorizer` returns the term-frequency matrix, not as a `DataFrame` or even as a `numpy` array, but as a `scipy` sparse matrix. A _sparse matrix_ is one whose entries are mostly zeroes. For example,

$$ \begin{pmatrix} 0 & 0 & 0 & 0 & 0 \\ 1.7 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & -0.8 & 0 \end{pmatrix} $$

is an example of a sparse matrix. Instead of storing 20 values (most of which are equal to 0), we can simply store the locations of the non-zero entries and their values:

- $(1, 0) \rightarrow 1.7$
- $(3, 3) \rightarrow -0.8$

All other entries of the matrix are assumed to be zero. This representation offers substantial memory savings when there are only a few non-zero entries. (But if not, then this representation can actually be more expensive.) Term-frequency matrices are usually sparse because most words do not appear in all documents.

The `scipy` sparse matrix format is used to store sparse matrices. If necessary, a `scipy` sparse matrix can be converted to a `numpy` matrix using the `.todense()` method.

In [13]:
tf_sparse.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 3, 1, 1]])

We can further convert this `numpy` matrix to a `pandas` `DataFrame`. To make the column names descriptive, we call the `.get_feature_names()` method of the `CountVectorizer`, which returns a list of the words in the order that they appear in the matrix.

In [14]:
pd.DataFrame(
    tf_sparse.todense(),
    columns=vec.get_feature_names()
)

,12,56,98,able,about,act,afraid,after,afternoon,again,...,yop,yopp,you,young,your,yourself,yourselves,zans,zeds,zeep
0,0,0,0,0,0,0,0,0,0,0,...,0,0,34,0,0,0,0,0,0,0
1,0,0,0,0,3,0,0,1,0,0,...,0,0,34,0,8,0,0,0,0,0
2,0,0,0,0,2,0,0,0,0,0,...,0,0,8,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,2,0,0,...,0,0,2,0,0,0,0,0,0,0
4,1,1,0,1,1,0,0,4,2,1,...,0,3,47,5,7,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,2,1,0,0,0,0,0,0
6,0,0,1,0,1,1,2,0,0,0,...,0,0,85,0,20,2,0,0,0,0
7,0,0,0,0,1,0,0,0,0,2,...,1,0,24,0,9,0,0,3,1,1


The term-frequency matrix that `CountVectorizer` produced is not exactly the same as the matrix that we produced ourselves using just `pandas`. Although the two matrices have the same number of rows (8, corresponding to the number of documents in the corpus), they have a different number of columns. It appears that `CountVectorizer` had a vocabulary that was 11 words smaller (1344 words instead of 1355). We can determine exactly which 11 words these are, by taking the set difference:

In [15]:
set(tf.columns) - set(vec.get_feature_names())

{'3', '4', '6', 'a', 'd', 'i', 'j', 'm', 'o', 's', 't'}

We see that all of the words that `CountVectorizer` missed were one-character long. By default, `CountVectorizer` only retains words that are at least 2 characters long. This behavior can be customized using the `token_pattern=` parameter, but we will not pursue that here, since 1-letter words are usually not useful for analysis anyway.

`CountVectorizer` can even count $n$-grams. If we wanted both unigrams (i.e., individual words) and bigrams, then we would specify `ngram_range=(1, 2)`. If we wanted only the bigrams, then we would specify `ngram_range=(2, 2)`. 

Let's get unigrams, bigrams, and trigrams.

In [16]:
vec = CountVectorizer(ngram_range=(1, 3))
vec.fit(docs_seuss)
vec.transform(docs_seuss)

<8x14918 sparse matrix of type '<class 'numpy.int64'>'
	with 16560 stored elements in Compressed Sparse Row format>

In [17]:
# number of non-zero values in the sparse matrix.
vec.transform(docs_seuss).count_nonzero()

16560

There are nearly 15,000 bigrams. If we wanted to store this data in a `DataFrame`, we would need as many columns, even though only about 16,000 out of the nearly 120,000 entries are nonzero. This is why sparse matrices are vital in text processing.

## TF-IDF

The problem with term frequencies (TF) is that common words like "the" and "that" tend to have high counts and dominate. A better indicator of whether two documents are similar is if they share rare words. For example, the word "eat" only appears in two of the Dr. Seuss stories. The presence of that word in two documents is a strong indicator that the documents are similar, so we should give more weight to terms like it.

This is the idea behind TF-IDF. We take each term frequency and re-weight it according to how many documents that term appears in (i.e., the **document frequency**). Since we want words that appear in fewer documents to get more weight, we take the **inverse document frequency** (IDF).  We take the logarithm of IDF because the distribution of IDFs is heavily skewed to the right. (Remember the discussion about transforming data from Chapter 1.4.) So in the end, the formula for IDF is:

$$ \textrm{idf}(t, D) = \log \frac{\text{# of documents}}{\text{# of documents containing $t$}} = \log \frac{|D|}{|d \in D: t \in d|}. $$

(Sometimes, $1$ will be added to the denominator to prevent division by zero, if there are terms in the vocabulary that do not appear in the corpus.)

To calculate TF-IDF, we simply multiply the term frequencies by the inverse document frequencies:

$$ \textrm{tf-idf}(d, t, D) = \textrm{tf}(d, t) \cdot \textrm{idf}(t, D). $$

Notice that unlike TF, the TF-IDF representation of a given document depends on the entire corpus of documents.

### Implementation from Scratch

Let's first see how to calculate TF-IDF from scratch, using the term-frequency matrix we obtained above.

In [18]:
# Get document frequencies 
# (How many documents does each word appear in?)
df = (tf > 0).sum(axis=0)
df

i        8
am       4
sam      1
that     8
do       7
        ..
swish    1
gack     1
park     1
clark    1
zeep     1
Length: 1355, dtype: int64

In [20]:
import numpy as np

# Get IDFs
idf = np.log(len(tf) / df)
idf.sort_values()

i           0.000000
a           0.000000
in          0.000000
they        0.000000
be          0.000000
              ...   
days        2.079442
gown        2.079442
insisted    2.079442
kites       2.079442
zeep        2.079442
Length: 1355, dtype: float64

In [21]:
# Calculate TF-IDFs
tf_idf = tf * idf
tf_idf

,i,am,sam,that,do,not,like,you,green,eggs,...,zeds,upon,heads,haircut,wave,swish,gack,park,clark,zeep
green_eggs_and_ham,0.0,11.090355,39.509389,0.0,4.807130,0.0,5.875381,0.0,20.794415,20.794415,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cat_in_the_hat,0.0,0.000000,0.000000,0.0,3.338285,0.0,1.869439,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fox_in_socks,0.0,0.000000,0.000000,0.0,1.068251,0.0,0.133531,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
hop_on_pop,0.0,0.693147,0.000000,0.0,0.000000,0.0,0.801188,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
horton_hears_a_who,0.0,0.693147,0.000000,0.0,0.934720,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
how_the_grinch_stole_christmas,0.0,0.000000,0.000000,0.0,0.534126,0.0,0.267063,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
oh_the_places_youll_go,0.0,0.000000,0.000000,0.0,0.534126,0.0,0.133531,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
one_fish_two_fish,0.0,2.079442,0.000000,0.0,1.602377,0.0,2.804159,0.0,0.000000,0.000000,...,2.079442,2.079442,2.079442,2.079442,2.079442,6.238325,4.158883,2.079442,2.079442,2.079442


### Implementation using `scikit-learn`

We will not generally implement TF-IDF from scratch, like we did above. Instead, we will use Scikit-Learn's `TfidfVectorizer`, which operates similarly to `CountVectorizer`, except that it returns a matrix of the TF-IDF weights.

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(docs_seuss) # This determines the vocabulary.
tf_idf_sparse = vec.transform(docs_seuss)
tf_idf_sparse

<8x1344 sparse matrix of type '<class 'numpy.float64'>'
	with 2308 stored elements in Compressed Sparse Row format>

## Cosine Similarity

We now have a representation of each text document as a vector of numbers. Each number can either be a term frequency or a TF-IDF weight. We can visualize each vector as an arrow in a high-dimensional space, where each dimension represents a word. The magnitude of the vector along a dimension represents the "frequency" (TF or TF-IDF) of that word in the document. For example, if our vocabulary only contains two words, "i" and "sam", then the arrows shown below might represent two documents:

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space.png?raw=1" width="300"/>

To fit $k$-nearest neighbors or $k$-means clustering, we need some way to measure the distance between two documents (i.e., two vectors). We could use Euclidean distance, as we have done in the past.

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space_euclidean.png?raw=1" width="300"/>

But Euclidean distance does not make sense for TF or TF-IDF vectors. To see why, consider the two documents:

1. "I am Sam." 
2. "I am Sam. Sam I am." 

The two documents are obviously very similar. But the vector for the second is twice as long as the vector for the first because the second document has twice as many occurrences of each word. This is true whether we use TF or TF-IDF weights. If we calculate the Euclidean distance between these two vectors, then they will seem quite far apart.

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space_example.png?raw=1" width="300"/>

With TF and TF-IDF vectors, the distinguishing property is their _direction_. Because the two vectors above point in the same direction, they are similar. We need a distance metric that measures how different their directions are. A natural way to measure the difference between the directions of two vectors is the angle between them.

<img src="https://github.com/dlsun/pods/blob/master/10-Textual-Data/vector_space_cosine.png?raw=1" width="300"/>

The cosine of the angle between two vectors ${\bf a}$ and ${\bf b}$ can be calculated as:

$$ \cos \theta = \frac{\sum a_j b_j}{\sqrt{\sum a_j^2} \sqrt{\sum b_j^2}}. $$

Although it is possible to work out the angle $\theta$ from this formula, it is more common to report $\cos\theta$ as a measure of similarity between two vectors. This similarity metric is called **cosine similarity**. Notice that when the angle $\theta$ is close to 0 (i.e., when the two vectors point in nearly the same direction), the value of $\cos\theta$ is high (close to 1.0, which is the maximum possible value).

The cosine _distance_ is defined as 1 minus the similarity. This makes it so that 0 means that the two vectors point in the same direction:

$$ d_{\cos}({\bf a}, {\bf b}) = 1 - \cos(\theta({\bf a}, {\bf b})) = 1 - \frac{\sum a_j b_j}{\sqrt{\sum a_j^2} \sqrt{\sum b_j^2}}. $$

### Implementation from Scratch

Let's calculate the cosine similarity between document 0 (_Green Eggs and Ham_) and document 2 (_Fox in Socks_) using the TF-IDF representation.

In [23]:
# Calculate the numerator.
a = tf_idf_sparse[0, :]
b = tf_idf_sparse[2, :]
dot = a.multiply(b).sum()

# Calculate the terms in the denominator.
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(b.multiply(b).sum())

# Cosine similarity is their ratio.
dot / (a_len * b_len)

0.10197809112431884

These two vectors are not very similar, as evidenced by their low cosine similarity (close to 0.0). Let's try to find the most similar documents in the corpus to _Green Eggs and Ham_---in other words, its nearest neighbors. To do this, we will take advantage of _broadcasting_: we will multiply a TF-IDF vector (representing document 0) by the entire TF-IDF matrix and calculate the sum over the columns. This will give us a vector of dot products.

In [24]:
# Calculate the numerators.
a = tf_idf_sparse[0, :]
B = tf_idf_sparse
dot = a.multiply(B).sum(axis=1)
dot

matrix([[34282.66548742],
        [13856.74242973],
        [ 3192.73842529],
        [ 1662.65737991],
        [ 8698.41557824],
        [ 5098.5714281 ],
        [ 6918.05569539],
        [ 6958.3624518 ]])

In [25]:
# Calculate the denominators.
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(B.multiply(B).sum(axis=1))
print(a_len)
b_len

185.15578707514322


matrix([[185.15578708],
        [220.67601821],
        [169.09048515],
        [ 67.77450402],
        [246.13363988],
        [208.51785767],
        [151.24831788],
        [151.8499661 ]])

In [26]:
# Calculate their ratio to obtain cosine similarities.
dot / (a_len * b_len)

matrix([[1.        ],
        [0.33913196],
        [0.10197809],
        [0.13249489],
        [0.19086746],
        [0.13205899],
        [0.2470337 ],
        [0.24748852]])

Now let's put this matrix into a `DataFrame` so that we can easily sort the values in descending order.

In [27]:
cos_similarities = pd.DataFrame(dot / (a_len * b_len))[0]
most_similar = cos_similarities.sort_values(ascending=False)
most_similar

0    1.000000
1    0.339132
7    0.247489
6    0.247034
4    0.190867
3    0.132495
5    0.132059
2    0.101978
Name: 0, dtype: float64

Of course, the most similar document in the corpus to _Green Eggs and Ham_ (with a perfect cosine similarity of 1.0) is itself. But the next most similar text is _The Cat in the Hat_.

### Implementation using scikit-learn

It is also possible to calculate cosine similarities/distances in `scikit-learn` using the same API that we used to calculate distances in Chapter 3.

In [33]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

CSM = pd.DataFrame(cosine_similarity(tf_idf_sparse),index=docs_seuss.index,columns=docs_seuss.index)
CSM

,green_eggs_and_ham,cat_in_the_hat,fox_in_socks,hop_on_pop,horton_hears_a_who,how_the_grinch_stole_christmas,oh_the_places_youll_go,one_fish_two_fish
green_eggs_and_ham,1.000000,0.339132,0.101978,0.132495,0.190867,0.132059,0.247034,0.247489
cat_in_the_hat,0.339132,1.000000,0.190015,0.332097,0.588969,0.539433,0.442517,0.613000
fox_in_socks,0.101978,0.190015,1.000000,0.097093,0.171208,0.128882,0.154492,0.193159
hop_on_pop,0.132495,0.332097,0.097093,1.000000,0.254134,0.214191,0.147176,0.368720
horton_hears_a_who,0.190867,0.588969,0.171208,0.254134,1.000000,0.594556,0.485144,0.480848
how_the_grinch_stole_christmas,0.132059,0.539433,0.128882,0.214191,0.594556,1.000000,0.300953,0.377783
oh_the_places_youll_go,0.247034,0.442517,0.154492,0.147176,0.485144,0.300953,1.000000,0.404552
one_fish_two_fish,0.247489,0.613000,0.193159,0.368720,0.480848,0.377783,0.404552,1.000000


In [35]:
CSM.stack().drop_duplicates().sort_values()

fox_in_socks                    hop_on_pop                        0.097093
green_eggs_and_ham              fox_in_socks                      0.101978
fox_in_socks                    how_the_grinch_stole_christmas    0.128882
green_eggs_and_ham              how_the_grinch_stole_christmas    0.132059
                                hop_on_pop                        0.132495
hop_on_pop                      oh_the_places_youll_go            0.147176
fox_in_socks                    oh_the_places_youll_go            0.154492
                                horton_hears_a_who                0.171208
cat_in_the_hat                  fox_in_socks                      0.190015
green_eggs_and_ham              horton_hears_a_who                0.190867
fox_in_socks                    one_fish_two_fish                 0.193159
hop_on_pop                      how_the_grinch_stole_christmas    0.214191
green_eggs_and_ham              oh_the_places_youll_go            0.247034
                         

The $(i, j)$th entry of this matrix represents the cosine similarity between the $i$th and $j$th documents. So the first row of this matrix contains the similarities between _Green Eggs and Ham_ and the other documents in the corpus. Check that these numbers match the ones we obtained manually.

In [29]:
cosine_similarity(tf_idf_sparse)[0]

array([1.        , 0.33913196, 0.10197809, 0.13249489, 0.19086746,
       0.13205899, 0.2470337 , 0.24748852])

# Exercises

1\. Suppose we had instead compared documents using cosine similarity on the term frequencies (TF), instead of TF-IDF. Does this change the conclusion?

In [37]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

CSM = pd.DataFrame(cosine_similarity(tf_sparse),index=docs_seuss.index,columns=docs_seuss.index)
CSM.stack().drop_duplicates().sort_values()

green_eggs_and_ham              how_the_grinch_stole_christmas    0.196255
                                hop_on_pop                        0.209853
                                fox_in_socks                      0.219427
fox_in_socks                    hop_on_pop                        0.253080
hop_on_pop                      oh_the_places_youll_go            0.253759
green_eggs_and_ham              horton_hears_a_who                0.286314
fox_in_socks                    how_the_grinch_stole_christmas    0.293803
                                oh_the_places_youll_go            0.315316
hop_on_pop                      how_the_grinch_stole_christmas    0.321074
green_eggs_and_ham              one_fish_two_fish                 0.344958
                                oh_the_places_youll_go            0.353771
fox_in_socks                    one_fish_two_fish                 0.374255
                                horton_hears_a_who                0.377986
cat_in_the_hat           

2\. Suppose we had instead used Euclidean distance on the TF-IDF weights, instead of cosine distance. Does this change the conclusion? What if we first normalize the length of the TF-IDF vector for each document before calculating Euclidean distance?

_Challenge Exercise:_ Can you prove the above fact mathematically?

In [47]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, cosine_distances

ESM1 = pd.DataFrame(euclidean_distances(tf_idf_sparse),index=docs_seuss.index,columns=docs_seuss.index)
ESM1.stack().drop_duplicates().sort_values(ascending=False).head(10)

green_eggs_and_ham              horton_hears_a_who                278.330025
fox_in_socks                    horton_hears_a_who                273.719560
green_eggs_and_ham              how_the_grinch_stole_christmas    259.933106
how_the_grinch_stole_christmas  green_eggs_and_ham                259.933106
cat_in_the_hat                  fox_in_socks                      251.214995
how_the_grinch_stole_christmas  fox_in_socks                      250.964003
fox_in_socks                    how_the_grinch_stole_christmas    250.964003
hop_on_pop                      horton_hears_a_who                238.110143
green_eggs_and_ham              fox_in_socks                      237.673686
                                cat_in_the_hat                    235.089527
dtype: float64

In [48]:
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

CSM2 = pd.DataFrame(cosine_similarity(tf_idf_sparse),index=docs_seuss.index,columns=docs_seuss.index)
CSM2.stack().drop_duplicates().sort_values().head(10)

fox_in_socks        hop_on_pop                        0.097093
green_eggs_and_ham  fox_in_socks                      0.101978
fox_in_socks        how_the_grinch_stole_christmas    0.128882
green_eggs_and_ham  how_the_grinch_stole_christmas    0.132059
                    hop_on_pop                        0.132495
hop_on_pop          oh_the_places_youll_go            0.147176
fox_in_socks        oh_the_places_youll_go            0.154492
                    horton_hears_a_who                0.171208
cat_in_the_hat      fox_in_socks                      0.190015
green_eggs_and_ham  horton_hears_a_who                0.190867
dtype: float64

In [49]:
ESM1.stack().drop_duplicates().sort_values(ascending=False).tail(10)

cat_in_the_hat          oh_the_places_youll_go            205.022590
hop_on_pop              how_the_grinch_stole_christmas    204.985588
fox_in_socks            one_fish_two_fish                 204.281037
green_eggs_and_ham      hop_on_pop                        188.549023
fox_in_socks            hop_on_pop                        175.953381
cat_in_the_hat          one_fish_two_fish                 175.138540
oh_the_places_youll_go  one_fish_two_fish                 165.383568
hop_on_pop              oh_the_places_youll_go            156.371659
                        one_fish_two_fish                 141.641796
green_eggs_and_ham      green_eggs_and_ham                  0.000000
dtype: float64

In [50]:
CSM2.stack().drop_duplicates().sort_values(ascending=False).tail(10)

green_eggs_and_ham  horton_hears_a_who                0.190867
cat_in_the_hat      fox_in_socks                      0.190015
fox_in_socks        horton_hears_a_who                0.171208
                    oh_the_places_youll_go            0.154492
hop_on_pop          oh_the_places_youll_go            0.147176
green_eggs_and_ham  hop_on_pop                        0.132495
                    how_the_grinch_stole_christmas    0.132059
fox_in_socks        how_the_grinch_stole_christmas    0.128882
green_eggs_and_ham  fox_in_socks                      0.101978
fox_in_socks        hop_on_pop                        0.097093
dtype: float64

In [59]:
df_tf_idf = pd.DataFrame.sparse.from_spmatrix(tf_idf_sparse)
df_tf_idf_norm = df_tf_idf.divide(np.sqrt((df_tf_idf**2).sum(axis=1)),axis=0)
df_tf_idf_norm

,0,1,2,3,4,5,6,7,8,9,...,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.183629,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
1,0.000000,0.000000,0.000000,0.000000,0.019107,0.000000,0.000000,0.008206,0.000000,0.000000,...,0.00000,0.000000,0.154072,0.000000,0.050951,0.000000,0.000000,0.000000,0.00000,0.00000
2,0.000000,0.000000,0.000000,0.000000,0.016624,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.047312,0.000000,0.008312,0.000000,0.000000,0.000000,0.00000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053440,0.000000,0.000000,...,0.00000,0.000000,0.029510,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
4,0.010174,0.010174,0.000000,0.010174,0.005710,0.000000,0.000000,0.029430,0.020347,0.008526,...,0.00000,0.030521,0.190953,0.042632,0.039971,0.000000,0.010174,0.000000,0.00000,0.00000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.009592,0.010064,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
6,0.000000,0.000000,0.016556,0.000000,0.009292,0.016556,0.033112,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.561990,0.000000,0.185849,0.033112,0.000000,0.000000,0.00000,0.00000
7,0.000000,0.000000,0.000000,0.000000,0.009256,0.000000,0.000000,0.000000,0.000000,0.027641,...,0.01649,0.000000,0.158051,0.000000,0.083301,0.000000,0.000000,0.049471,0.01649,0.01649


In [62]:
np.sqrt((df_tf_idf_norm**2).sum(axis=1))

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
dtype: float64

In [64]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, cosine_distances

ESM2 = pd.DataFrame(euclidean_distances(df_tf_idf_norm),index=docs_seuss.index,columns=docs_seuss.index)
ESM2.stack().drop_duplicates().sort_values(ascending=False).head(10)

fox_in_socks        hop_on_pop                        1.343806
green_eggs_and_ham  fox_in_socks                      1.340166
fox_in_socks        how_the_grinch_stole_christmas    1.319938
green_eggs_and_ham  how_the_grinch_stole_christmas    1.317529
                    hop_on_pop                        1.317198
hop_on_pop          oh_the_places_youll_go            1.306004
fox_in_socks        oh_the_places_youll_go            1.300391
                    horton_hears_a_who                1.287472
cat_in_the_hat      fox_in_socks                      1.272780
green_eggs_and_ham  horton_hears_a_who                1.272110
dtype: float64

In [67]:
CSM2.stack().drop_duplicates().sort_values().head(10)

fox_in_socks        hop_on_pop                        0.097093
green_eggs_and_ham  fox_in_socks                      0.101978
fox_in_socks        how_the_grinch_stole_christmas    0.128882
green_eggs_and_ham  how_the_grinch_stole_christmas    0.132059
                    hop_on_pop                        0.132495
hop_on_pop          oh_the_places_youll_go            0.147176
fox_in_socks        oh_the_places_youll_go            0.154492
                    horton_hears_a_who                0.171208
cat_in_the_hat      fox_in_socks                      0.190015
green_eggs_and_ham  horton_hears_a_who                0.190867
dtype: float64

In [65]:
ESM1.stack().drop_duplicates().sort_values(ascending=False).head(10)

green_eggs_and_ham              horton_hears_a_who                278.330025
fox_in_socks                    horton_hears_a_who                273.719560
green_eggs_and_ham              how_the_grinch_stole_christmas    259.933106
how_the_grinch_stole_christmas  green_eggs_and_ham                259.933106
cat_in_the_hat                  fox_in_socks                      251.214995
how_the_grinch_stole_christmas  fox_in_socks                      250.964003
fox_in_socks                    how_the_grinch_stole_christmas    250.964003
hop_on_pop                      horton_hears_a_who                238.110143
green_eggs_and_ham              fox_in_socks                      237.673686
                                cat_in_the_hat                    235.089527
dtype: float64

3\. Convert the self-summary variable (`essay0`) in the OKCupid data set (https://dlsun.github.io/pods/data/okcupid.csv) to a TF-IDF representation. Use this to find a match for user 61 based on what he says he is looking for in a partner (`essay9`).

The [data dictionary](https://github.com/rudeboybert/JSE_OkCupid/blob/master/okcupid_codebook.txt) may help you understand what the columns mean.

In [68]:
df_okcupid = pd.read_csv('https://dlsun.github.io/pods/data/okcupid.csv')
df_okcupid.head()

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,...,location,offspring,orientation,pets,religion,sex,sign,smokes,height,status
0,31,NaN,mostly vegetarian,socially,sometimes,graduated from college/university,"75% nice, 45% shy, 80% stubborn, 100% charming...",i'm a new nurse. it rules.,"multiple-choice questions, dancing.",it depends on the people.,...,"san francisco, california",might want kids,gay,likes cats,buddhism,f,taurus and it&rsquo;s fun to think about,no,67.0,single
1,25,average,NaN,socially,NaN,working on college/university,"i like trees, spending long periods of time co...","studying landscape horticulture, beekeeping, g...","wasting time, making breakfast, nesting",i have a lot of freckles,...,"oakland, california",NaN,gay,NaN,NaN,m,sagittarius and it&rsquo;s fun to think about,no,66.0,single
2,43,curvy,NaN,rarely,never,graduated from masters program,NaN,NaN,NaN,NaN,...,"san francisco, california",has a kid,straight,likes dogs and has cats,other and laughing about it,f,leo and it&rsquo;s fun to think about,trying to quit,65.0,single
3,31,average,NaN,socially,never,NaN,"i am a seeker of laughs ,music ,magick good pe...",i strive to live life to the fullest and to tr...,i am good at my magic and weaving a world of i...,i am guessing y'all would notice my jewelry an...,...,"san francisco, california",doesn&rsquo;t want kids,gay,NaN,other and very serious about it,m,capricorn and it&rsquo;s fun to think about,trying to quit,70.0,single
4,34,NaN,NaN,socially,NaN,graduated from ph.d program,i've just moved here from london after finishi...,i'm doing a postdoc in psychology at stanford,NaN,NaN,...,"san francisco, california",NaN,gay,NaN,NaN,m,cancer but it doesn&rsquo;t matter,NaN,71.0,single


In [71]:
essays = df_okcupid['essay0'].append(df_okcupid['essay9']).fillna('')
essays

0       75% nice, 45% shy, 80% stubborn, 100% charming...
1       i like trees, spending long periods of time co...
2                                                        
3       i am a seeker of laughs ,music ,magick good pe...
4       i've just moved here from london after finishi...
                              ...                        
2995              you can woo a man with your vocabulary.
2996                                                     
2997                                                     
2998    you've got the know-how and the elbow grease t...
2999    you think we could enrich each others lives in...
Length: 6000, dtype: object

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(essays) # This determines the vocabulary.
#tf_idf_sparse = vec.transform(essays)
#tf_idf_sparse
tf_idf_sparse0 = pd.DataFrame.sparse.from_spmatrix(vec.transform(df_okcupid['essay0'].fillna('')))
tf_idf_sparse0.index = df_okcupid['essay0'].index
tf_idf_sparse9 = pd.DataFrame.sparse.from_spmatrix(vec.transform(df_okcupid['essay9'].fillna('')))
tf_idf_sparse9.index = df_okcupid['essay9'].index
tf_idf_sparse9.loc[[61]]

CSM_okcupid = pd.DataFrame(cosine_similarity(tf_idf_sparse9.loc[[61]],tf_idf_sparse0),index=[61])#,columns=docs_seuss.index)
CSM_okcupid.stack().drop_duplicates().sort_values(ascending=False).tail(10)

61  349     0.005385
    1549    0.005344
    1176    0.005336
    2435    0.004986
    605     0.004282
    1301    0.004112
    1925    0.003554
    1963    0.003258
    2466    0.003203
    2       0.000000
dtype: float64

In [81]:
CSM_okcupid.stack().drop_duplicates().sort_values(ascending=False).head(10)

61  2344    0.241772
    2678    0.215790
    959     0.212673
    1936    0.211059
    342     0.208499
    862     0.208170
    1859    0.204973
    1117    0.202926
    257     0.201132
    224     0.199141
dtype: float64

Exercises 4-5 ask you to work with the Enron spam data set (https://dlsun.github.io/pods/data/enron_spam.csv). This data set contains the subjects and bodies of a sample of e-mails that the Federal Energy Regulatory Commission (FERC) collected during their 2002 investigation of the energy company Enron. 

In [82]:
df_spam = pd.read_csv('https://dlsun.github.io/pods/data/enron_spam.csv')
df_spam

,subject,body,spam
0,mirant 4 / 01,we invoiced mirant americas for deal 705989 an...,0
1,re : lobo payout,because the payback was done for october 2001 ...,0
2,entex transaction 7,"for december 1999 , since the volumes for tran...",0
3,re : hpl transport contracts,i would think that the first contract should g...,0
4,welcome to aol instant messenger !,welcome to the aol instant messenger ( sm ) se...,0
...,...,...,...
1985,NaN,"discount meds right from home\nvalium , xanax ...",1
1986,reduce monthly - payments,"thank you for your mor tg age application , wh...",1
1987,reply soon ! ! !,"dear sir ,\ni know this email will reach you a...",1
1988,fwd : need | xianax ? vl @ gra % v + a + lium ...,we give you the power to make an educated choi...,1


4\. Each e-mail has additionally been manually labeled as spam (1) or not (0). Find the TF-IDF representation of the bodies of the e-mails. Which e-mail was most similar to e-mail 0 (not spam)? Which e-mail was most similar to e-mail 1001 (spam)?

In [83]:
corpus = df_spam['body'].fillna('')
corpus

0       we invoiced mirant americas for deal 705989 an...
1       because the payback was done for october 2001 ...
2       for december 1999 , since the volumes for tran...
3       i would think that the first contract should g...
4       welcome to the aol instant messenger ( sm ) se...
                              ...                        
1985    discount meds right from home\nvalium , xanax ...
1986    thank you for your mor tg age application , wh...
1987    dear sir ,\ni know this email will reach you a...
1988    we give you the power to make an educated choi...
1989    you have [ 1 ] new message .\nplease view it a...
Name: body, Length: 1990, dtype: object

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(corpus) # This determines the vocabulary.
tf_idf_sparse = pd.DataFrame.sparse.from_spmatrix(vec.transform(corpus))

CSM_spam = pd.DataFrame(cosine_similarity(tf_idf_sparse.loc[[0,1001]],tf_idf_sparse.drop([0,1001])),index=[0,1001])#,columns=docs_seuss.index)
CSM_spam.stack().drop_duplicates().sort_values(ascending=False).tail(10)

1001  1744    0.001125
0     1852    0.000810
1001  1728    0.000785
      1842    0.000654
      1644    0.000635
      1505    0.000558
      1973    0.000554
0     1232    0.000297
1001  1401    0.000229
0     65      0.000000
dtype: float64

In [88]:
CSM_spam.stack().drop_duplicates().sort_values(ascending=False).loc[1001]

1663    0.573494
1097    0.500739
1120    0.448894
1985    0.437058
1010    0.427921
          ...   
1842    0.000654
1644    0.000635
1505    0.000558
1973    0.000554
1401    0.000229
Length: 1695, dtype: float64

In [87]:
CSM_spam.stack().drop_duplicates().sort_values(ascending=False).loc[0]

541     0.323298
326     0.238129
664     0.224681
544     0.215916
470     0.211796
          ...   
1644    0.001325
1505    0.001298
1852    0.000810
1232    0.000297
65      0.000000
Length: 1737, dtype: float64

5\. (This exercise requires material from Part II of this book.) Write a function `predict_spam()` that accepts an e-mail body and predicts whether or not it is spam using $k$-nearest neighbors on the Enron spam data set. Use cosine distance ($= 1 - \text{cosine similarity}$) as your distance metric.

Use your model to predict whether an e-mail with the body "free cash" is spam or not.

In [89]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3,metric='cosine')
neigh.fit(tf_idf_sparse, df_spam['spam'])

KNeighborsClassifier(metric='cosine', n_neighbors=3)

In [93]:
def predict_spam(text):
    return neigh.predict(vec.transform([text]))[0]

In [94]:
predict_spam('free cash')

1

In [99]:
predict_spam('What did you have for dinner')

1

In [100]:
predict_spam('What did free cash have for dinner')

0